In [21]:

import speech_recognition as sr

# Initialiser le recogniteur
recognizer = sr.Recognizer()

# Utiliser le microphone comme source audio
with sr.Microphone() as source:
    print("Parlez maintenant...")
    audio = recognizer.listen(source)

    try:
        # Reconnaissance vocale en utilisant Google Web Speech API
        text = recognizer.recognize_google(audio, language="fr-FR")
        print("Vous avez dit : " + text)
        # Export the text in /tmp/speech_recognition.txt
        with open("/tmp/speech_recognition.txt", "w") as f:
            f.write(text)
    except sr.UnknownValueError:
        print("Google Web Speech API n'a pas pu comprendre l'audio")
    except sr.RequestError as e:
        print("La requête vers Google Web Speech API a échoué; {0}".format(e))


Parlez maintenant...
Vous avez dit : bonjour J'aimerais procéder à un trajet de Paris pour aller à Amiens ça regarde-moi ça


In [10]:
import spacy
import time
from spacy.training.example import Example
import json

# Load training data
with open("training/spaCyFromToTraining.json", "r") as json_file:
   training_data = json.load(json_file)

# Load validation data
with open("validation/spaCyFromToValidation.json", "r") as json_file:
   validation_data = json.load(json_file)

# Load the model
nlp = spacy.load("fr_core_news_sm")

# Initialize the NER component
ner = nlp.get_pipe("ner")

# Add labels to the NER component
for _, annotations in training_data:
   for ent in annotations.get("entities"):
       ner.add_label(ent[2])

# Function to check if two entities match
def entity_match(pred_text, pred_start, pred_end, pred_label, true_start, true_end, true_label):
   return abs(pred_start - true_start) <= 2 and \
          abs(pred_end - true_end) <= 2 and \
          pred_label == true_label

# Train the model by epoch
for i in range(20): # Change the number of epochs according to your needs
   start_time = time.time()
   for text, annotations in training_data:
       # Create a spaCy Example
       example = Example.from_dict(nlp.make_doc(text), annotations)
       # Update the model with the training example
       nlp.update([example], drop=0.5)
   elapsed_time = time.time() - start_time

   # Evaluate the model on the validation set
   correct_predictions = 0
   total_predictions = 0
   for text, annotations in validation_data:
       doc = nlp(text)
       predicted_entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
       true_entities = annotations["entities"]
       for pred_text, pred_start, pred_end, pred_label in predicted_entities:
           total_predictions += 1
           if any(entity_match(pred_text, pred_start, pred_end, pred_label, true[0], true[1], true[2]) for true in true_entities):
               correct_predictions += 1
   accuracy = correct_predictions / total_predictions

   print(f"Epoch: {i}, Accuracy: {accuracy}, Time: {elapsed_time} seconds")

# Save the updated model
nlp.to_disk("models/test/")


Epoch: 0, Accuracy: 0.4046242774566474, Time: 0.9231841564178467 seconds
Epoch: 1, Accuracy: 0.3523809523809524, Time: 0.9112780094146729 seconds
Epoch: 2, Accuracy: 0.5306122448979592, Time: 0.9263458251953125 seconds
Epoch: 3, Accuracy: 0.6113989637305699, Time: 1.1015348434448242 seconds
Epoch: 4, Accuracy: 0.7329842931937173, Time: 0.9716470241546631 seconds
Epoch: 5, Accuracy: 0.7589743589743589, Time: 0.8939340114593506 seconds
Epoch: 6, Accuracy: 0.797979797979798, Time: 0.9421510696411133 seconds
Epoch: 7, Accuracy: 0.8080808080808081, Time: 0.9364850521087646 seconds
Epoch: 8, Accuracy: 0.8061224489795918, Time: 0.938319206237793 seconds
Epoch: 9, Accuracy: 0.8121827411167513, Time: 0.8902640342712402 seconds
Epoch: 10, Accuracy: 0.8186528497409327, Time: 0.9367609024047852 seconds
Epoch: 11, Accuracy: 0.8223350253807107, Time: 0.9521999359130859 seconds
Epoch: 12, Accuracy: 0.8527918781725888, Time: 0.9784502983093262 seconds
Epoch: 13, Accuracy: 0.855, Time: 0.91878771781921

In [20]:
import spacy

# Load the trained spaCy model
nlp = spacy.load("models/test")

# Example text for prediction
# Load the text from /tmp/speech_recognition.txt
with open("/tmp/speech_recognition.txt", "r") as f:
    text_to_predict = f.read()

# Process the text using the loaded model
doc = nlp(text_to_predict.lower())

# Extract named entities from the processed document
entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

# Print the extracted entities
print("Extracted Entities:")
for entity in entities:
    print(entity)

Extracted Entities:
('toulouse', 94, 102, 'TO')
('caen', 118, 122, 'TO')


In [12]:
from faker import Faker
import random
import json
import pandas as pd
fake = Faker("fr_FR")  # French locale
df = pd.read_csv("./format_src/format_data_station_city.csv", sep=";")
cities = df["City"].tolist()
def generate_training_data(nbr_data: int):
    training_data = []

    for _ in range(nbr_data):
        from_city = random.choice(cities)
        to_city = random.choice(cities)

        # Ensure unique cities
        while to_city == from_city:
            to_city = random.choice(cities)

        if random.choice([True, False]):
            from_city, to_city = to_city, from_city
        
        is_order_1 = random.choice([True, False, True])

        sentence_template_order_1 = random.choice([
            "Je veux voyager de {} à {}.",
            "il faut que j'aille à {}, je suis à {}",
            "je dois faire le trajet {} {}",
            "{} {}",
            "je suis à {} et j'aimerai allez à {}",
            "de {} comment se rendre à {}",
            "je suis perdu, je pense etre pas loin de {} et je dois allez à {}",
            "depuis hier je pense allez à {}, mais j'ai peur du trajet depuis {}",
            "gare de {} gare de {}"
            

        ])

        sentence_template_order_2 = random.choice([
            "j'aimme les Pommes de Amiens, mais je vais à {} de {}",
            "j'adore {}, comment y allez de {}",
            "je dois me de rendre à {} mais je suis à {}",
            "c'est super {}, comment y allez de {}",
            "je suis pas loin de Paris mais je dois allez à {} depuis {}"

        ])


        sentence = sentence_template_order_1.format(from_city, to_city) if is_order_1 else sentence_template_order_2.format(to_city, from_city)
        
        
        start_from = sentence.find(from_city)
        start_to = sentence.find(to_city)

        entities = [
                (start_from, start_from + len(from_city), "FROM"),
                (start_to, start_to + len(to_city), "TO")
        ]
        training_data.append((sentence, {"entities": entities}))

    return training_data


training_data = generate_training_data(100)

with open("training/spaCyFromToTraining.json", "w+") as file:
    json.dump(training_data, file)






In [13]:
def generate_validation_data(nbr_data: int):
    validation_data = []

    df = pd.read_csv("./format_src/format_data_station_city.csv", sep=";")
    cities = df["City"].tolist()
    for _ in range(nbr_data):
        from_city = random.choice(cities)
        to_city = random.choice(cities)

        # Ensure unique cities
        while to_city == from_city:
            to_city = random.choice(cities)

        if random.choice([True, False]):
            from_city, to_city = to_city, from_city
        
        is_order_1 = random.choice([True, False, True])

        sentence_template_order_1 = random.choice([
            "Je veux voyager de {} à {}.",
            "il faut que j'aille à {}, je suis à {}",
            "je dois faire le trajet {} {}",
            "{} {}",
            "je suis à {} et j'aimerai allez à {}",
            "de {} comment se rendre à {}",
            "je suis perdu, je pense etre pas loin de {} et je dois allez à {}",
            "depuis hier je pense allez à {}, mais j'ai peur du trajet depuis {}",
            "gare de {} gare de {}"
            
        ])

        sentence_template_order_2 = random.choice([
            "j'aimme les Pommes de Amiens, mais je vais à {} de {}",
            "j'adore {}, comment y allez de {}",
            "je dois me de rendre à {} mais je suis à {}",
            "c'est super {}, comment y allez de {}",
            "je suis pas loin de Paris mais je dois allez à {} depuis {}"

        ])

        sentence = sentence_template_order_1.format(from_city, to_city) if is_order_1 else sentence_template_order_2.format(to_city, from_city)
        
        start_from = sentence.find(from_city)
        start_to = sentence.find(to_city)

        # Check if the cities are found in the sentence
        if start_from == -1 or start_to == -1:
            continue

        # Check if the indices match the actual positions of the cities in the sentence
        if sentence[start_from:start_from+len(from_city)] != from_city or sentence[start_to:start_to+len(to_city)] != to_city:
            continue

        entities = [
                (start_from, start_from + len(from_city), "FROM"),
                (start_to, start_to + len(to_city), "TO")
        ]
        validation_data.append((sentence, {"entities": entities}))

    return validation_data


validation_data = generate_validation_data(100)

with open("validation/spaCyFromToValidation.json", "w+") as file:
    json.dump(validation_data, file)


In [14]:
import pandas as pd
import numpy


def extract_unique_station_and_cities():
    df = pd.read_csv('./src/liste-des-gares.csv', sep=';')
    

   
    station_list = numpy.array([])
    city_list = numpy.array([])

    for i in range(len(df)):
        row = df.iloc[i]

        # Traitement des données
        station = row[1].lower()
        city = row[7].lower()

        station_list = numpy.append(station_list, station)
        city_list = numpy.append(city_list, city)
    

    df_res = pd.DataFrame({
        "Station": station_list,
        "City": city_list
    })

    df_res.to_csv('./format_src/format_data_station_city.csv', sep=";")

    
    

extract_unique_station_and_cities()

/var/folders/_z/51xkbm4139b206gcdc8fxpww0000gn/T/ipykernel_1606/152893444.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  station = row[1].lower()
/var/folders/_z/51xkbm4139b206gcdc8fxpww0000gn/T/ipykernel_1606/152893444.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city = row[7].lower()
